# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
full_data_rows_list = [] 

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [5]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### colums:
0 artist  <br>
1 firstName of user  <br>
2 gender of user <br>
3 item number in session  <br>
4 last name of user  <br>
5 length of the song  <br>
6 level (paid or free song)  <br>
7 location of the user  <br>
8 sessionId  <br>
9 song title  <br>
10 userId

In [9]:
quer1 = "select artist, song_title, song_length from music_library WHERE sessionId = 338 and item_nb_session = 4"

query2 = "select artist, song_title, last_name_of_user from music_library WHERE sessionId = 182 and userId = 10"

query3 = "select firstName_of_user, last_name_of_user  WHERE song = 'All Hands Against His Own'"                    

# First query

In [10]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_session"
query = query + "(sessionID int,\
                  item_nb_session int,\
                  artist text,\
                  song text,\
                  length float,\
                  PRIMARY KEY (sessionID, item_nb_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into song_info_by_session \
                   (sessionID,\
                   item_nb_session,\
                   artist,\
                   song,\
                   length\
                   )"
        query = query + " Values (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)


**Task**:<br>
Query the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4<br>

**Description**: <br>
The following table shows the artist, song title and length of the session id 338 and item in session 4.<br>

**Which columns will be primary keys and why?**<br>
item nb session and session id is a compound primary key, as defined during creation process. Both together form a unique key. 

**What clustering keys am I using and why?**<br>
Essentially, the clustering columns determine the on-disk sort order within each partition. In this cases, the first column is the partition key (item nb) and the second column is the clustering column (session id). This combination of the partition key and clustering column composes a composite/compound primary key. <br>

**Why did I choose this specific table structure?**<br>
Basically, the creation of this table with the partionining of our data with its primary key is designed to serve the query.

**How is this structure benefitting the query output?**<br>
The structure of the primary key allows to filter the table given any session id and item nb and outputs every music app history. 


In [12]:
query = "select artist, song, length from song_info_by_session where sessionId = 338 and item_nb_session = 4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [13]:
from prettytable import PrettyTable
x = PrettyTable()

x.field_names = ["Artist", "Song", "Length"]
x.add_row(["Faithless", "Music Matters (Mark Knight Dub)", 495.30731201171875])


print(x)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


# Second query

In [14]:
query = "CREATE TABLE IF NOT EXISTS artist_info_by_session_and_user"
query = query + "(userID int,\
                  sessionID int,\
                  itemInSession int,\
                  artist text,\
                  song text,\
                  user_firstName text,\
                  user_lastName text,\
                  PRIMARY KEY ((userID, sessionID), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into artist_info_by_session_and_user \
                  (userID,\
                   sessionID,\
                   itemInSession,\
                   artist,\
                   song,\
                   user_firstName,\
                   user_lastName\
                  )"

        query = query + " Values (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)         

**Task**:<br>
Query name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182<br>

**Description**: <br>
The following table shows three columns: artist, song name and the user's first and last name. It is filtered by session id 182 und user id 10<br>

**Which columns will be primary keys and why?**<br>
Session id and user id is a compound primary key, as defined during creation process. Both together form a unique key. 

**What clustering keys am I using and why?**<br>
Essentially, the clustering columns determine the on-disk sort order within each partition. In this cases, the first column is the partition key (item nb) and the second column is the clustering column (session id). This combination of the partition key and clustering column composes a composite/compound primary key. Furthermore, an other clustering column is added, Item in Session, to ensure the on-disk sort order within each partition <br>

**Why did I choose this specific table structure?**<br>
Basically, the creation of this table with the partionining of our data with its primary and clustering key is designed to serve the query.

**How is this structure benefitting the query output?**<br>
The structure of the primary key and clustering key allows to filter the table given any session id and user id. The result is sorted by item in session. 


In [16]:
query = "select artist, song, user_firstName, user_lastName from artist_info_by_session_and_user where sessionId = 182 and userID = 10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user_firstname, row.user_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [17]:
from prettytable import PrettyTable
x = PrettyTable()

x.field_names = ["Artist", "Song", "User name"]
x.add_row(["Down To The Bone", "Keep On Keepin", "Sylvie Cruz"])
x.add_row(["Three Drives Greece", "2000", "Sylvie Cruz"])
x.add_row(["Lonnie Gordon", "Catch You Baby (Steve Pitron & Max Sanna Radio Edit)", "Sylvie Cruz"])


print(x)

+---------------------+------------------------------------------------------+-------------+
|        Artist       |                         Song                         |  User name  |
+---------------------+------------------------------------------------------+-------------+
|   Down To The Bone  |                    Keep On Keepin                    | Sylvie Cruz |
| Three Drives Greece |                         2000                         | Sylvie Cruz |
|    Lonnie Gordon    | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz |
+---------------------+------------------------------------------------------+-------------+


# Third query

In [ ]:
query = "CREATE TABLE IF NOT EXISTS user_info_by_song"
query = query + "(song text,\
                  userId int,\
                  user_firstName text,\
                  user_lastName text,\
                  PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into user_info_by_song \
                  (song,\
                   userId,\
                   user_firstName,\
                   user_lastName\
                  )"  
        query = query + " Values (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4], ))
        except Exception as e:
            print(e)

 **Task**:<br>
Query every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own<br>

**Description**: <br>
The following table shows all users listened to the song 'All Hands Against His Own<br>

**Which columns will be primary keys and why?**<br>
Song and user id is a compound primary key, as defined during creation process. Both together form a unique key. 

**What clustering keys am I using and why?**<br>
Essentially, the clustering columns determine the on-disk sort order within each partition. In this case no specification is needed. <br>

**Why did I choose this specific table structure?**<br>
Basically, the creation of this table with the partionining of our data with its primary key is designed to serve the query.

**How is this structure benefitting the query output?**<br>
The structure of the primary key allows to filter the table given any song name and outputs every user who listened to this song. 


In [ ]:
query = "select user_firstName, user_lastName from user_info_by_song where song ='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_firstname, row.user_lastname)

In [ ]:
from prettytable import PrettyTable
x = PrettyTable()

x.field_names = ["User name"]
x.add_row(["Jacqueline Lynch"])
x.add_row(["Tegan Levine"])
x.add_row(["Sara Johnson"])


print(x)

### Drop the tables before closing out the sessions

In [22]:
query = "drop table song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table artist_info_by_session_and_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_info_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()